# Feature Engineering

#### Import the required libraries

In [39]:
# Pandas for DataFrames
import pandas as pd
pd.set_option('display.max_columns', 100)

# Matplotlib for visualization
from matplotlib import pyplot as plt

# display plots in the notebook
%matplotlib inline 

# Seaborn for visualization
import seaborn as sns

#### Import the cleaned dataset

In [ ]:
# Load cleaned dataset from the previous lecture
df = pd.read_csv('cleaned_df.csv')
df.head()

# Quick EDA hack

!pip install pandas-profiling

#importing required packages
import pandas_profiling

#descriptive statistics
pandas_profiling.ProfileReport(df)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1882 entries, 0 to 1881
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   price          1882 non-null   int64 
 1   year_sold      1882 non-null   int64 
 2   property_tax   1882 non-null   int64 
 3   insurance      1882 non-null   int64 
 4   beds           1882 non-null   int64 
 5   baths          1882 non-null   int64 
 6   sqft           1882 non-null   int64 
 7   year_built     1882 non-null   int64 
 8   lot_size       1882 non-null   int64 
 9   basement       1882 non-null   int64 
 10  property_type  1882 non-null   object
dtypes: int64(10), object(1)
memory usage: 161.9+ KB


## I. Domain Knowledge

#### A. Popular Properties

2 bedroom and 2 bathroom properties are especially popular for investors. Let's create an indicator variable just for properties with 2 beds and 2 baths.

Build your code step by step. Here's how:

* Your first condition `df.beds == 2`
* Your second condition `df.baths == 2`
* Combine the two conditions with an `&` operator
* Convert the resulting data to `int` type.
* Store the result in a column called **popular**

In [50]:
# Check the boolean conditions of 2 beds and 2 baths, convert them into 0 and 1 by 'astype'
(df.beds== 2) & (df.baths== 2)

0       False
1       False
2       False
3       False
4       False
        ...  
1877    False
1878    False
1879    False
1880    False
1881    False
Length: 1882, dtype: bool

In [58]:
# Create indicator variable for properties with 2 beds and 2 baths
df['popular']= ((df.beds== 2) & (df.baths== 2)).astype(int)

In [59]:
df.head()

,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement,property_type,popular
0,295850,2013,234,81,1,1,584,2013,0,0,Condo,0
1,216500,2006,169,51,1,1,612,1965,0,1,Condo,0
2,279900,2012,216,74,1,1,615,1963,0,0,Condo,0
3,379900,2005,265,92,1,1,618,2000,33541,0,Condo,0
4,340000,2002,88,30,1,1,634,1992,0,0,Condo,0


In [60]:
# Check how many properties have 2 baths and 2 beds 
df.popular.value_counts()

0    1704
1     178
Name: popular, dtype: int64

#### B. Housing Market Recession

We are modeling housing prices in the United States, it's important to consider the housing market recession around 2008. According to data from Zillow, the lowest housing prices were from 2010 to end of 2013.

<br>
Create an indicator feature **recession**

Here's how:
* Your first condition `year_sold >= 2010`
* Your second condition `year_sold <= 2013`
* Combine the two conditions with an `&` operator
* Convert the resulting data to `int` type.
* Store the result in a column called **recession**

In [61]:
# Create a new variable recession
df['recession'] = ((df.year_sold >= 2010) & (df.year_sold <= 2013)).astype(int)

In [62]:
# Check how many propoerties were sold during recession period 
df.head()

,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement,property_type,popular,recession
0,295850,2013,234,81,1,1,584,2013,0,0,Condo,0,1
1,216500,2006,169,51,1,1,612,1965,0,1,Condo,0,0
2,279900,2012,216,74,1,1,615,1963,0,0,Condo,0,1
3,379900,2005,265,92,1,1,618,2000,33541,0,Condo,0,0
4,340000,2002,88,30,1,1,634,1992,0,0,Condo,0,0


## II. Interaction Features

In the first step, you engineered features from domain knowledge. interaction features can be products, sums, or differences between two features.

#### A. Property Age

We have the features `year_sold` and the `year_built`. let's create a new feature `property_age`

In [63]:
# Create a property age feature
df['property_age'] = df.year_sold - df.year_built
df.head()

,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement,property_type,popular,recession,property_age
0,295850,2013,234,81,1,1,584,2013,0,0,Condo,0,1,0
1,216500,2006,169,51,1,1,612,1965,0,1,Condo,0,0,41
2,279900,2012,216,74,1,1,615,1963,0,0,Condo,0,1,49
3,379900,2005,265,92,1,1,618,2000,33541,0,Condo,0,0,5
4,340000,2002,88,30,1,1,634,1992,0,0,Condo,0,0,10


Do a quick sanity check on that feature. Run `df.describe()` and check the stats for the feature `property_age`

In [65]:
# Do you see any error?
df.property_age.describe()

count    1882.000000
mean       24.126461
std        21.153271
min        -8.000000
25%         6.000000
50%        20.000000
75%        38.000000
max       114.000000
Name: property_age, dtype: float64

In [72]:
# Check number of observations with 'property_age' < 0
(df.property_age < 0).value_counts()

False    1863
Name: property_age, dtype: int64

On second thought, this could be an error or that some homeowners buy houses before the construction company builts them. But for the purpose of this project we will remove these observations.

We'll do a quick ad-hoc data cleaning and remove these observations from our dataset.

#### Remove observations where `property_age` is less than 0.
* Keep only observations where `property_age` is 0 and above.

In [71]:
# Remove rows where property_age is less than 0
# if you want to use .drop()
#df.drop(index=df[df.property_age<0].index, inplace=True)
df= df[df.property_age>=0]
df

,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement,property_type,popular,recession,property_age
0,295850,2013,234,81,1,1,584,2013,0,0,Condo,0,1,0
1,216500,2006,169,51,1,1,612,1965,0,1,Condo,0,0,41
2,279900,2012,216,74,1,1,615,1963,0,0,Condo,0,1,49
3,379900,2005,265,92,1,1,618,2000,33541,0,Condo,0,0,5
4,340000,2002,88,30,1,1,634,1992,0,0,Condo,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,760000,2008,881,249,5,5,7842,2003,436035,1,Bunglow,0,0,5
1878,690000,2015,1553,473,5,6,6501,1956,23086,1,Bunglow,0,0,59
1879,600000,1999,942,287,5,6,7064,1995,217800,1,Bunglow,0,0,4
1880,759900,2009,803,245,5,6,7500,2006,8886,1,Bunglow,0,0,3


## III. Dummy Variables

Most Machine Learning algorithms cannot handle text data. Therefore, we need to create what are called **dummy variables** for all our categorical features.
<br><br>
Pandas has an easy function for doing this automatically.
* `pd.get_dummies()`

In [75]:
# Create dummy variables for 'exterior_walls', 'roof', 'property_type'
df= pd.get_dummies(df, columns= ['property_type'], drop_first= True)
df.head(2)

,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement,popular,recession,property_age,property_type_Condo
0,295850,2013,234,81,1,1,584,2013,0,0,0,1,0,1
1,216500,2006,169,51,1,1,612,1965,0,1,0,0,41,1


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1863 entries, 0 to 1881
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   price                1863 non-null   int64
 1   year_sold            1863 non-null   int64
 2   property_tax         1863 non-null   int64
 3   insurance            1863 non-null   int64
 4   beds                 1863 non-null   int64
 5   baths                1863 non-null   int64
 6   sqft                 1863 non-null   int64
 7   year_built           1863 non-null   int64
 8   lot_size             1863 non-null   int64
 9   basement             1863 non-null   int64
 10  popular              1863 non-null   int32
 11  recession            1863 non-null   int32
 12  property_age         1863 non-null   int64
 13  property_type_Condo  1863 non-null   uint8
dtypes: int32(2), int64(11), uint8(1)
memory usage: 191.0 KB


### Save the final dataset

We will save this dataset and train our model on it.

Use pandas function `.to_csv()` and remember to set index=None

In [77]:
# Save the data as 'final.csv'
df.to_csv('final.csv', index=None)